In [8]:
#packages
#python 2/3 compatibility
from __future__ import absolute_import, division, print_function, unicode_literals
#packages
import pathlib
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from keras.models import model_from_json
from tensorflow.keras.models import model_from_json

In [9]:
#load gas model
json_file = open('predGasModel.json','r')
loaded_model_jsonGAS = json_file.read()
json_file.close()
loaded_modelGAS = model_from_json(loaded_model_jsonGAS)

#load weights into new model
loaded_modelGAS.load_weights("predAnnualGas.h5")
print("Loaded Gas Model from disk")

Loaded Gas Model from disk


In [10]:
#load gas model scaler weights
scalerGas = joblib.load("trainPredGasScaler.save") 

In [11]:
#load pred. savings model
json_file = open('predSaveModel.json','r')
loaded_model_jsonPrdSave = json_file.read()
json_file.close()
loaded_modelPrdSave = model_from_json(loaded_model_jsonPrdSave)

#load weights into new model
loaded_modelPrdSave.load_weights("predSaveDollar.h5")
print("Loaded Savings Model from disk")

Loaded Savings Model from disk


In [12]:
#load savings model scaler weights
scalerSaveRate = joblib.load("trainPredSaveScaler.save") 

In [ ]:
#Gas Input Data Format/features
# 'SF', 'Floors','Year Built', 'Value', 'E annual'
#Savings Input Data Format
# 'SF', 'Floors','Year Built', 'Value', 'E annual', 'G annual'
#then normalized to train datasets

In [13]:
#load test dataset
testData = pd.read_csv('delawarePwrConcat.csv')

In [16]:
testData.head()

,ownerName,Value,Year Built,Floors,SF,service_address,residentName,E annual
0,PRICE AMOS E & CONSTANCE D,146400,1911,1.0,1046,"8724 WEISER RD WALDO, OH 43356",PRICE AMOS JR,11607
1,BARROWS TIMOTHY A,156400,1961,1.0,1362,"8739 WEISER RD WALDO, OH 43356",BARROWS TIMOTHY A,13642
2,SHONYA FARMS LLC,154400,2018,1.5,1946,"8695 WEISER RD WALDO, OH 43356",HEMANS KURT,9786
3,WISEMAN CLARA MAE,84400,1953,1.0,1120,"80 NORTON RD WALDO, OH 43356",WISEMAN CLARA M,12507
4,BROWN RICHARD A & LOIS E,48200,1901,1.0,1188,"151 NORTON RD WALDO, OH 43356",ISAACS NORMA,10655


In [15]:
testData.rename(columns = {'OWNER1':'ownerName', 'MARKET_TOT': 'Value', 'YRBUILT': 'Year Built',
                           'STORYHGT': 'Floors', 'TOT_FIN_L_': 'SF' ,'customer_name': 'residentName'}, inplace = True) 

In [17]:
testDataFeatures = testData[['SF', 'Floors','Year Built', 'Value', 'E annual']]

In [18]:
testDataFeatures.head()

,SF,Floors,Year Built,Value,E annual
0,1046,1.0,1911,146400,11607
1,1362,1.0,1961,156400,13642
2,1946,1.5,2018,154400,9786
3,1120,1.0,1953,84400,12507
4,1188,1.0,1901,48200,10655


In [20]:
#select a single row to pass to model to validate on
testOneGas = testDataFeatures[0:1]

In [21]:
#Function to normalize test data value to dist. range from train data
def norm(x, scaler):
    return (x - scaler['mean']) / scaler['std']

In [22]:
#load test GAS data
#define how to input the values here
#testGas = 'some input type here'
testGas = testOneGas

In [23]:
#normalize test gas data
testGasNorm = norm(testGas, scalerGas)

In [24]:
#compile loaded GAS model
optimizerGAS = tf.keras.optimizers.RMSprop(0.001)
loaded_modelGAS.compile(loss='mean_squared_logarithmic_error', optimizer=optimizerGAS, metrics=['mae', 'mse'])

In [25]:
#get gas prediction
outputGasPredVal = loaded_modelGAS.predict(testGasNorm)
print(outputGasPredVal)

[[1031.6674]]


In [28]:
#load test SAVINGS data
#define how to input the values here
#testSaveRate = 'some input type here'
testSaveRate = testOneGas.copy()

In [33]:
#add gas prediction to savings data
testSaveRate['G annual']  = outputGasPredVal.astype(int)

In [34]:
#view test data
print(testSaveRate)

     SF  Floors  Year Built   Value  E annual  G annual
0  1046     1.0        1911  146400     11607      1031


In [35]:
#normalize test savings data
testSaveRateNorm = norm(testSaveRate, scalerSaveRate)

In [37]:
#compile loaded SAVINGS model
optimizerSAVE = tf.keras.optimizers.RMSprop(0.001)
loaded_modelPrdSave.compile(loss='mean_squared_logarithmic_error', optimizer=optimizerSAVE, metrics=['mae', 'mse'])

In [38]:
#pass data to savings model, get prediction
outputSavePredVal = loaded_modelPrdSave.predict(testSaveRateNorm)
print(outputSavePredVal)

[[651.42847]]


In [39]:
#add savings prediction to test data
testSaveRate['Est_Savings']  = outputSavePredVal.astype(int)

In [47]:
#view test data
#testSaveRate

In [42]:
#elec data x local value
elecMltprVal = 0.13

In [43]:
#gas data x local value
gasMltprVal = 8.18

In [44]:
#combine values to calc. annual dollar elec. and gas cost contd..
#est. annual total utility cost before and after savings and savings percentage
#returns savings rate as pct
#calc annual energy dollar val for central Ohio
energyLocYrCost = testSaveRate['E annual'] * elecMltprVal
#calc annual gas dollar val for central Ohio
gasLocYrCost = testSaveRate['G annual'] * gasMltprVal
#calc est. annual combined cost of gas/elec. utilities
totActUtil = int(energyLocYrCost + gasLocYrCost)
#est new annual utility cost less pred. savings value
newEstUtil = int(totActUtil - outputSavePredVal)
#converts savings value to percentage value
estPctSavings = int((outputSavePredVal/totActUtil) * 100)

In [53]:
#return all new prediction values as list that can be parsed
#output format: [current annual util. dollar value, new annual util. dollar value, est. annual savings dollar amount, est. savings rate as percentage]
outputDataVal = [totActUtil, newEstUtil, outputSavePredVal, estPctSavings]

#for demo purposes, prints values below, uncomment to run
print("Current Total Annual Utility Cost-USD: %d" % (outputDataVal[0]))
print("Estimated New Annual Utility Cost-USD: %d" % (outputDataVal[1]))
print("Estimated Annual Savings Amount-USD: %d" % (outputDataVal[2]))
print("Estimated Annual Savings Percent-percentage: %d" % (outputDataVal[3]))

Current Total Annual Utility Cost-USD: 9942
Estimated New Annual Utility Cost-USD: 9290
Estimated Annual Savings Amount-USD: 651
Estimated Annual Savings Percent-percentage: 6


In [ ]:
#define how you want to push output data to be used/returned to user